## Import MatPlotLib & Setup

In [ ]:
# Setup plotting
import matplotlib.pyplot as plt
# Set Matplotlib defaults
plt.rc('figure', autolayout=True)
plt.rc('axes', labelweight='bold', labelsize='large',
       titleweight='bold', titlesize=18, titlepad=10)
plt.rc('animation', html='html5')

## Import Keras, Pandas & other used libraries

In [ ]:
import pandas as pd

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.impute import SimpleImputer
from sklearn.pipeline import make_pipeline
from sklearn.compose import make_column_transformer
from sklearn.model_selection import GroupShuffleSplit

from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras import callbacks

## Import Data

In [ ]:
df_game_events = pd.read_csv("../data/game_events.csv")
game_events_features = ['timestamp', 'event_type', 'killer_id', 'killer_name', 'killer_team_id',]

df_game_metadata = pd.read_csv("../data/game_metadata.csv")
game_metadata_features = ['game_id']

df_game_players_stats = pd.read_csv("../data/game_players_stats.csv")
df_game_players_stats.head()

game_players_stats_features_num = ['game_length', 'champion_name', 'tower_kills', 'inhibitor_kills', 'dragon_kills', 'herald_kills', 'baron_kills', 
                               'player_kills', 'player_deaths', 'player_assists','total_minions_killed', 'gold_earned', 'level', 
                               'total_damage_dealt','total_damage_dealt_to_champions', 'total_damage_taken', 'wards_placed', 
                               'largest_killing_spree', 'largest_multi_kill']

game_players_stats_features_by_user_num = ['player_kills', 'player_deaths', 'player_assists', 'total_minions_killed', 'gold_earned', 'level', 
                                            'total_damage_dealt', 'total_damage_dealt_to_champions', 'total_damage_taken', 'wards_placed', 
                                            'largest_killing_spree', 'largest_multi_kill']
game_players_stats_features_by_user_cat = ['champion_name']

pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
match_players = df_game_players_stats[df_game_players_stats['game_id'] == 0]
match_players.head(10)

## Preprocessing data for win determination with one player's data

In [ ]:
def get_useful_player_data(player_id: int, game_id: int) -> pd.DataFrame:
    """
    Get player data for a specific player and game.
    """
    # Filter by game_id first
    game_data = df_game_players_stats[df_game_players_stats['game_id'] == game_id]
        
    # Filter by player_id
    player_data = game_data[game_data['player_id'] == player_id]
    
    # Calculate total kills in the match (across both teams)
    total_kills = game_data['player_kills'].sum()
    
    # Create a new DataFrame with only the selected columns
    player_selected = player_data[game_players_stats_features_num].copy()
    
    # Add the total kills column
    player_selected['total_match_kills'] = total_kills
    
    return player_selected

# Possibly add worthless death ratio & free kill ration, objective contest winrate & loserate
representative_player_data_col_num = ['KLA', 'gold_pm', 'lvl_pm', 'creep_score_pm', 'wards_pm', 
               'damage_dealt_tkr', 'damage_dealt_pgr', 'damage_taken_tkr', 'damage_taken_pgr',
               'largest_multikill', 'largest_killing_spree_tkr']

def get_final_input_features_performances(dataframe : pd.DataFrame) -> pd.DataFrame:
    
    new_df = pd.DataFrame()
    
    new_df['KLA'] = (dataframe['player_kills'] + dataframe['player_assists']) / (dataframe['player_deaths'] + 1)
    new_df['gold_pm'] = dataframe['gold_earned'] / dataframe['game_length']
    new_df['lvl_pm'] = dataframe['level'] / dataframe['game_length']
    new_df['creep_score_pm'] = dataframe['total_minions_killed'] / dataframe['game_length']
    new_df['wards_pm'] = dataframe['wards_placed'] / dataframe['game_length']
    new_df['damage_dealt_tkr'] = dataframe['total_damage_dealt'] / dataframe['total_match_kills']
    new_df['damage_dealt_pgr'] = new_df['damage_dealt_tkr'] / (dataframe['gold_earned'])
    new_df['damage_taken_tkr'] = dataframe['total_damage_taken'] / dataframe['total_match_kills']
    new_df['damage_taken_pgr'] = new_df['damage_taken_tkr'] / (dataframe['gold_earned'])
    new_df['largest_multikill'] = dataframe['largest_multi_kill']
    new_df['largest_killing_spree_tkr'] = dataframe['largest_killing_spree'] / dataframe['total_match_kills']
    
    return new_df

win_loose_preprocessor = make_column_transformer(
    (StandardScaler(), representative_player_data_col_num)
)

top_X = df_game_players_stats.loc[df_game_players_stats['role'] == player_roles[0]].copy()
top_X = top_X.loc[top_X['player_id'] != 21].copy()
top_y = top_X['win'].copy()
top_group = top_X['game_id'].copy()

top_X = top_X[game_players_stats_features_by_user_num]

def group_split(X: pd.DataFrame, y: pd.Series, group, train_size=0.75):
    splitter = GroupShuffleSplit(train_size=train_size)
    train, test = next(splitter.split(X, y, groups=group))
    return (X.iloc[train], X.iloc[test], y.iloc[train], y.iloc[test])

top_X_train, top_X_valid, top_y_train, top_y_valid = group_split(top_X, top_y, top_group, train_size=0.75)

top_X_train = win_loose_preprocessor.fit_transform(top_X_train)
top_X_valid = win_loose_preprocessor.transform(top_X_valid)

top_input_shape = [top_X_train.shape[1]]
print("Input shape: {}".format(top_input_shape))